In [41]:
from pyfuseki import FusekiQuery, FusekiUpdate
import pysolr  
from api.src.schemas.settings import Settings
import httpx

In [57]:
settings = Settings()
collectionQuery = FusekiQuery(f'{settings.url}:3030', 'collection')
collectionUpdate = FusekiUpdate(f'{settings.url}:3030', 'collection')

#SOLR
solr = pysolr.Solr(f'{settings.url}:8983/solr/acervo/', timeout=10)

TEST JORNEY OF HUMANITY

Assunto

In [24]:
# Import LOC
url = 'http://localhost:8000/import/loc/subjects?uri=http%3A%2F%2Fid.loc.gov%2Fauthorities%2Fsubjects%2Fsh85040817'
response = httpx.get(url)
response

<Response [200 OK]>

In [30]:
uri = 'http://id.loc.gov/authorities/subjects/sh85040804'
url = f'http://localhost:8000/import/loc/subjects?uri={uri}'
response = httpx.get(url, timeout=120)
response

<Response [200 OK]>

In [31]:
obj = response.json()

In [32]:
# POST ASSUNTO
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/subject/', headers=headers, json=obj)
response.status_code

201

Agent

In [34]:
url = 'http://localhost:8000/import/loc/agents?uri=http%3A%2F%2Fid.loc.gov%2Fauthorities%2Fnames%2Fn86870523'
response = httpx.get(url, timeout=120)
response


<Response [200 OK]>

WORK

In [5]:
url = 'http://localhost:8000/import/loc/works?uri=http%3A%2F%2Fid.loc.gov%2Fresources%2Fworks%2F22600263'
response = httpx.get(url, timeout=120)
response

<Response [200 OK]>

In [6]:
obj = response.json()

In [7]:
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.classification import Classification
from api.src.schemas.metadata.bibframe.contribution import Contribution
from api.src.schemas.metadata.bibframe.element import Element
from api.src.schemas.metadata.bibframe.title import Title

class Work(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    content: list[Element]
    language: list[Element]
    title: Title
    classification: Optional[Classification]
    contribution: Optional[list[Contribution]]
    subject: Optional[list[Element]]   
    genreForm: Optional[list[Element]]
    note: Optional[str]
    summary: Optional[str]
    tableOfContents: Optional[str]
    supplementaryContent: Optional[list[Element]]
    illustrativeContent: Optional[list[Element]]
    intendedAudience: Optional[list[Element]]
    geographicCoverage: Optional[list[Element]]

In [13]:
obj['subject'] = [{'label': 'Desenvolvimento econômico',
  'lang': 'pt',
  'uri': 'https://bibliokeia.com/authorities/Topic/bka-2',
  'type': ['http://id.loc.gov/ontologies/bibframe/Topic']},

 {'label': 'História econômica',
  'lang': 'pt',
  'uri': 'https://bibliokeia.com/authorities/Topic/bka-1',
  'type': ['http://id.loc.gov/ontologies/bibframe/Topic']},

 {'label': 'Igualdade',
  'lang': 'pt',
  'uri': 'https://bibliokeia.com/authorities/Topic/bka-4',
  'type': ['http://id.loc.gov/ontologies/bibframe/Topic']}]

In [14]:
request = Work(**obj)
request

Work(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 6, 30), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 30, 10, 26, 5, 265649), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='22600263')], status=Status(value='mstatus:new', label='novo')), type=['Work', 'Text', 'Monograph'], content=[Element(label='text', lang=None, uri='http://id.loc.gov/vocabulary/contentTypes/txt', type=['http://id.loc.gov/ontologies/bibframe/Content'])], language=[Element(label='English', lang='en', uri='http://id.loc.gov/vocabulary/languages/eng', type=['http://id.loc.gov/ontologies/bibframe/Language

In [ ]:
# POST ASSUNTO
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/cataloguing/work', headers=headers, json=obj)
response.status_code

In [ ]:
def DocWork(request, id):

    doc = {
        "id": id,
        "type": request.type,
        "content": [i.label for i in request.content],
        "mainTitle": request.title.mainTitle,
        'language': [i.label for i in request.language],
        "subtitle": request.title.subtitle,
        "cdd": request.classification.classificationPortion,
        "note": request.note,
        "summary": request.summary,
        "tableOfContents": request.tableOfContents,
        "supplementaryContent": [i.label for i in request.supplementaryContent] if request.supplementaryContent else None,
        "illustrativeContent": [i.label for i in request.illustrativeContent] if request.illustrativeContent else None,
        "intendedAudience": [i.label for i in request.intendedAudience] if request.intendedAudience else None,
        "geographicCoverage": [i.label for i in request.geographicCoverage] if request.geographicCoverage else None,
        }
    
    if request.contribution:
        contributions = list()
        for i in request.contribution:
            c = { "id": f"{id}/contribution/{i.agent.split('/')[-1]}",
                "type": [i.split('/')[-1] for i in i.type],
                "agent": i.agent,
                "label": i.label,
                "role": i.role } 
            contributions.append(c)
        doc['contribution'] = contributions   

    if request.subject:
        pass
    if request.genreForm:
        pass

    responseSolr =  solr.add([doc], commit=True)

    return responseSolr 

responseSolr = DocWork(request, 'bkc-1')
responseSolr

In [76]:
solr.add([doc], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":153}}\n'

Instance


In [19]:
uri = 'http://id.loc.gov/resources/instances/22600263'
url = f'http://localhost:8000/import/loc/instances?uri={uri}'
instance = httpx.get(url)
instance

<Response [200 OK]>

In [21]:
objInstance = instance.json()


In [23]:
objInstance.keys()

dict_keys(['adminMetadata', 'type', 'title', 'carrier', 'copyrightDate', 'dimensions', 'extent', 'instanceOf', 'issuance', 'media', 'provisionActivity', 'provisionActivityStatement', 'responsibilityStatement', 'seriesStatement'])

In [25]:
objInstance['instanceOf'] = {'label': 'Galor, Oded, 1953- The journey of humanity',
 'lang': None,
 'uri': 'https://bibliokeia.com/resources/work/bkc-1',
 'type': ['http://id.loc.gov/ontologies/bibframe/Work']}

In [27]:
# POST INSTANCE
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/cataloguing/instance', headers=headers, json=objInstance, timeout=120)
response.status_code

201

ITEMS

In [31]:
objItem = {
  "itemOf": {
    "label": "The journey of humanity",
    "lang": "en",
    "uri": "https://bibliokeia.com/resources/instance/bkc-2",
    "type": [
      "Instance"
    ]
  },
  "items": [
    {
      "adminMetadata": {
        "identifiedBy": [
          {
            "type": "Local",
            "assigner": "http://id.loc.gov/vocabulary/organizations/brmninpa",
            "value": "string"
          }
        ]
      },
      "cdd": "542.5 ",
      "cutter": "O48j",
      "year": "2023",
      "shelf": "E1-P1",
      "barcode": "23-0001"
    }
  ]
}

In [30]:
from pydantic import BaseModel
from typing import List
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.element import Element

class Item(BaseModel):
    adminMetadata: AdminMetadata
    cdd: str
    cutter: str
    year: str
    shelf: str
    barcode: str

class Items_Schema(BaseModel):
  itemOf: Element
  items: List[Item]

In [37]:
from api.src.function.bibframe.Item.graphItem import MakeGraphItems

In [32]:
request = Items_Schema(**objItem)
request

Items_Schema(itemOf=Element(label='The journey of humanity', lang='en', uri='https://bibliokeia.com/resources/instance/bkc-2', type=['Instance']), items=[Item(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 6, 30), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 30, 10, 31, 39, 204364), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='string')], status=Status(value='mstatus:new', label='novo')), cdd='542.5 ', cutter='O48j', year='2023', shelf='E1-P1', barcode='23-0001')])

In [33]:
for item in request.items:
    print(item)

adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 6, 30), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 30, 10, 31, 39, 204364), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='string')], status=Status(value='mstatus:new', label='novo')) cdd='542.5 ' cutter='O48j' year='2023' shelf='E1-P1' barcode='23-0001'


In [34]:
item

Item(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 6, 30), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 30, 10, 31, 39, 204364), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='string')], status=Status(value='mstatus:new', label='novo')), cdd='542.5 ', cutter='O48j', year='2023', shelf='E1-P1', barcode='23-0001')

In [43]:
graph = MakeGraphItems(item, request.itemOf, "TESTE")
print(graph)

PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/> 

    INSERT DATA {
        GRAPH <https://bibliokeia.com/resources/item/TESTE>
        {
        <https://bibliokeia.com/resources/item/23-0001> a bf:Item ;
        bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:encodingLevel menclvl:f ;
            bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            bf:creationDate "2023-06-30"^^xsd:date ;  
            bf:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/isbd> ;
            bf:descriptionLanguage <http://id.loc.gov/vocabulary/languages/por> ;
            bf:generationProcess [ a 

In [44]:
r = collectionUpdate.run_sparql(graph)
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [45]:
from api.src.function.bibframe.Instance.hasItem import HasItem

In [47]:
request.itemOf.uri

'https://bibliokeia.com/resources/instance/bkc-2'

In [49]:
upInstance = HasItem(request.itemOf.uri, 'bkc-5')
print(upInstance)

PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        INSERT DATA
        { GRAPH <https://bibliokeia.com/resources/instance/bkc-2> 
        { <https://bibliokeia.com/resources/instance/bkc-2>  bf:hasItem  <https://bibliokeia.com/resources/item/bkc-5> } }


In [55]:
from rdflib import Graph

g = Graph()
g.parse('item.rdf')

<Graph identifier=Nf6b328f59c454894965b2ac159128add (<class 'rdflib.graph.Graph'>)>

In [56]:
g.serialize('testeNT.ttl')

<Graph identifier=Nf6b328f59c454894965b2ac159128add (<class 'rdflib.graph.Graph'>)>

In [60]:
ask = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    ASK { graph <https://bibliokeia.com/resources/work/bkc-1>
    { <https://bibliokeia.com/resources/work/bkc-1> bf:hasInstance ?o } }"""

response = collectionQuery.run_sparql(ask)
response = response.convert()



In [62]:
response['boolean']

True

In [63]:
id = 'bkc-1'
r = solr.search(q=f'id:{id}', **{'fl': '*,[child]'})
r

In [66]:
[doc] = r.docs
nMeta = ['id', ]
ids = list()
for k, v in doc.items():
    
        if k not in nMeta:
                print(k, v)
        #     if type(v) == list:
        #         for i in v:
        #             ids.append(i['id'])
        #     else:
        #         ids.append(v['id'])

id bkc-1
type ['Work', 'Text', 'Monograph']
content ['text']
mainTitle ['The journey of humanity']
language ['English']
cdd [338.9001]
summary ['"Why are humans the only species to have escaped - only very recently - the subsistence trap, allowing us to enjoy a standard of living that vastly exceeds all others? And why have we progressed so unequally around the world, resulting in the great disparities between nations that exist today? Immense in scope and packed with astounding connections, Galor\'s gripping narrative explains how technology, population size, and adaptation led to a stunning "phase change" in the human story a mere two hundred years ago. But by tracing that same journey back in time and peeling away the layers of influence - colonialism, political institutions, societal structure, culture - he arrives also at an explanation of inequality\'s ultimate causes: those ancestral populations that enjoyed fruitful geographical characteristics and rich diversity were set on th

In [1]:
def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    elif n == 0:
        return 1
    else:
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result

In [5]:
factorial(8)

40320